In [23]:
!pip install mysql-connector-python

     |████████████████████████████████| 9.3 MB 9.9 MB/s            
     |████████████████████████████████| 486 kB 70.4 MB/s            
You should consider upgrading via the '/Users/mino/Desktop/Git/CustomLectureProject/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

시간표(.xlsx)불러오기

In [5]:
df = pd.read_excel("종합강의시간표목록.xlsx") 

df.head()

/Users/mino/Desktop/Git/CustomLectureProject/venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


번호 학년/가진급학년  교과과정     교과영역구분      학수강좌번호       교과목명  교원명 수업캠퍼스  \
0   1    1,2학년  학문기초  제4영역:자연과학  PRI4001-01   미적분학및연습1  문환표    서울   
1   2    1,2학년  학문기초  제4영역:자연과학  PRI4001-02   미적분학및연습1  박장희    서울   
2   3    1,2학년  학문기초  제4영역:자연과학  PRI4002-01  일반물리학및실험1  조훈영    서울   
3   4    1,2학년  학문기초  제4영역:자연과학  PRI4012-01   미적분학및연습2  엄건애    서울   
4   5    1,2학년  학문기초  제4영역:자연과학  PRI4012-02   미적분학및연습2  최광호    서울   

                                       요일/시간  \
0                                        NaN   
1  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   
2                      월7.0-11.5/15:00-19:40   
3  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   
4  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   

                                                 강의실  ...  혁신교과목구분  원어강의  \
0                                                NaN  ...     MOOC   NaN   
1  102-352(학술/문화관 K352 352 강의실),102-352(학술/문화관 K3...  ...      NaN   NaN   
2                              307-309(명진관 A309 강의실)  ...      NaN   NaN   
3  407-109(정보문화관P P304 강의실),407-109(정보문화관P P304 강의실)  ...      NaN   NaN   
4  102-217(학술/문화관 K217 217 강의실),102-217(학술/문화관 K2...  ...      NaN   NaN   

   강평결과  이수구분 강의미리보기  개설대학        개설학과/전공 집중이수학기구분                비고  \
0   NaN    학기    NaN  이과대학       이과대학 수학과      NaN             사이버강의   
1   NaN    학기    NaN  이과대학       이과대학 수학과      NaN               NaN   
2   NaN    학기    NaN  이과대학  이과대학 물리반도체과학부      NaN     팀티칭, 수강정원 50명   
3   NaN    학기    NaN  이과대학       이과대학 수학과      NaN               NaN   
4   NaN    학기    NaN  이과대학       이과대학 수학과      NaN  공과대 일괄수강신청 예정 강좌   

                              교과목영문명  
0                          Calculus1  
1                          Calculus1  
2  General Physics and Experiments I  
3                          Calculus2  
4                          Calculus2  

[5 rows x 28 columns]

종합강의시간표에서 필요없는 컬럼 제거

In [6]:
drop_list = ["학년/가진급학년", "이론", "실습", "공학설계", "강의계획서"
             ,"교과목해설", "강의유형", "혁신교과목구분", "원어강의", "강평결과",
             "이수구분", "강의미리보기", "집중이수학기구분", "교과목영문명", "강의종류" ]

df.drop(columns=drop_list, inplace=True)
#df.set_index("번호", inplace=True)


df.head()

번호  교과과정     교과영역구분      학수강좌번호       교과목명  교원명 수업캠퍼스  \
0   1  학문기초  제4영역:자연과학  PRI4001-01   미적분학및연습1  문환표    서울   
1   2  학문기초  제4영역:자연과학  PRI4001-02   미적분학및연습1  박장희    서울   
2   3  학문기초  제4영역:자연과학  PRI4002-01  일반물리학및실험1  조훈영    서울   
3   4  학문기초  제4영역:자연과학  PRI4012-01   미적분학및연습2  엄건애    서울   
4   5  학문기초  제4영역:자연과학  PRI4012-02   미적분학및연습2  최광호    서울   

                                       요일/시간  \
0                                        NaN   
1  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   
2                      월7.0-11.5/15:00-19:40   
3  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   
4  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   

                                                 강의실  학점  개설대학        개설학과/전공  \
0                                                NaN   3  이과대학       이과대학 수학과   
1  102-352(학술/문화관 K352 352 강의실),102-352(학술/문화관 K3...   3  이과대학       이과대학 수학과   
2                              307-309(명진관 A309 강의실)   4  이과대학  이과대학 물리반도체과학부   
3  407-109(정보문화관P P304 강의실),407-109(정보문화관P P304 강의실)   3  이과대학       이과대학 수학과   
4  102-217(학술/문화관 K217 217 강의실),102-217(학술/문화관 K2...   3  이과대학       이과대학 수학과   

                 비고  
0             사이버강의  
1               NaN  
2     팀티칭, 수강정원 50명  
3               NaN  
4  공과대 일괄수강신청 예정 강좌

복합값인 '학수강좌번호' 분리해서 저장 (제1정규형)

In [7]:
df[['학수코드', '학수번호']] = df['학수강좌번호'].str.split('-', n=1, expand=True)

df.drop(columns=["학수강좌번호"], inplace=True)

df.head()

번호  교과과정     교과영역구분       교과목명  교원명 수업캠퍼스  \
0   1  학문기초  제4영역:자연과학   미적분학및연습1  문환표    서울   
1   2  학문기초  제4영역:자연과학   미적분학및연습1  박장희    서울   
2   3  학문기초  제4영역:자연과학  일반물리학및실험1  조훈영    서울   
3   4  학문기초  제4영역:자연과학   미적분학및연습2  엄건애    서울   
4   5  학문기초  제4영역:자연과학   미적분학및연습2  최광호    서울   

                                       요일/시간  \
0                                        NaN   
1  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   
2                      월7.0-11.5/15:00-19:40   
3  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   
4  월4.0-5.0/12:00-13:30,수3.0-4.0/11:00-12:30   

                                                 강의실  학점  개설대학        개설학과/전공  \
0                                                NaN   3  이과대학       이과대학 수학과   
1  102-352(학술/문화관 K352 352 강의실),102-352(학술/문화관 K3...   3  이과대학       이과대학 수학과   
2                              307-309(명진관 A309 강의실)   4  이과대학  이과대학 물리반도체과학부   
3  407-109(정보문화관P P304 강의실),407-109(정보문화관P P304 강의실)   3  이과대학       이과대학 수학과   
4  102-217(학술/문화관 K217 217 강의실),102-217(학술/문화관 K2...   3  이과대학       이과대학 수학과   

                 비고     학수코드 학수번호  
0             사이버강의  PRI4001   01  
1               NaN  PRI4001   02  
2     팀티칭, 수강정원 50명  PRI4002   01  
3               NaN  PRI4012   01  
4  공과대 일괄수강신청 예정 강좌  PRI4012   02

컬럼 순서 변경

In [8]:
print(df.columns.tolist())

['번호', '교과과정', '교과영역구분', '교과목명', '교원명', '수업캠퍼스', '요일/시간', '강의실', '학점', '개설대학', '개설학과/전공', '비고', '학수코드', '학수번호']


In [9]:
new_order = ['번호', '교과과정', '교과영역구분',  '학수코드', '학수번호',  '교과목명', '교원명', '수업캠퍼스', '요일/시간', '강의실', '학점', '개설대학', '개설학과/전공', '비고']
df = df.reindex(columns=new_order)

print(df.to_string(index=False))

  번호   교과과정     교과영역구분     학수코드 학수번호                                                           교과목명         교원명 수업캠퍼스                                                                                요일/시간                                                                                                                                                     강의실  학점     개설대학                           개설학과/전공                                                                                                  비고
   1   학문기초  제4영역:자연과학  PRI4001   01                                                       미적분학및연습1         문환표    서울                                                                                  NaN                                                                                                                                                     NaN   3     이과대학                          이과대학 수학과                                                                                               사이

다중값 제거(제4정규형) - 강의실

In [10]:
df_split = df[['번호', '강의실']]

df_split

번호                                                강의실
0        1                                                NaN
1        2  102-352(학술/문화관 K352 352 강의실),102-352(학술/문화관 K3...
2        3                              307-309(명진관 A309 강의실)
3        4  407-109(정보문화관P P304 강의실),407-109(정보문화관P P304 강의실)
4        5  102-217(학술/문화관 K217 217 강의실),102-217(학술/문화관 K2...
...    ...                                                ...
2615  2616                                                NaN
2616  2617                                                NaN
2617  2618                                                NaN
2618  2619                        408-229(정보문화관Q Q202 컴퓨터실습실)
2619  2620                        408-229(정보문화관Q Q202 컴퓨터실습실)

[2620 rows x 2 columns]

In [11]:
# '강의실' 컬럼의 값을 ','를 기준으로 분리하여 새로운 행으로 만들기
df_split = df_split.assign(강의실=df_split['강의실'].str.split(',')).explode('강의실', ignore_index=True)

df_split

번호                           강의실
0        1                           NaN
1        2  102-352(학술/문화관 K352 352 강의실)
2        2  102-352(학술/문화관 K352 352 강의실)
3        3         307-309(명진관 A309 강의실)
4        4      407-109(정보문화관P P304 강의실)
...    ...                           ...
3990  2616                           NaN
3991  2617                           NaN
3992  2618                           NaN
3993  2619   408-229(정보문화관Q Q202 컴퓨터실습실)
3994  2620   408-229(정보문화관Q Q202 컴퓨터실습실)

[3995 rows x 2 columns]

다중값 제거(제4정규형) - 시간

In [12]:
df_split2 = df[['번호', '요일/시간']]

df_split2 = df_split2.assign(요일시간=df_split2['요일/시간'].str.split(',')).explode('요일시간', ignore_index=True)

df_split2.drop(columns=['요일/시간'], inplace=True)

df_split2

번호                   요일시간
0        1                    NaN
1        2   월4.0-5.0/12:00-13:30
2        2   수3.0-4.0/11:00-12:30
3        3  월7.0-11.5/15:00-19:40
4        4   월4.0-5.0/12:00-13:30
...    ...                    ...
3770  2616                    NaN
3771  2617                    NaN
3772  2618                    NaN
3773  2619   월9.0-9.5/17:00-18:00
3774  2620   화9.0-9.5/17:00-18:00

[3775 rows x 2 columns]

In [13]:
# '요일'과 '시간'을 추출하는 함수 (NaN 값 예외 처리)
def extract_day_and_time(input_string):
    try:
        day = input_string[0]  # 첫 번째 문자가 요일 정보입니다.
        time = input_string[1:].split('/')[0]  # '/'로 분리하여 두 번째 부분이 시간 정보입니다.
        return day, time
    except:
        return np.NaN, np.NaN

In [14]:
df_split2['요일'] = df_split2['요일시간'].apply(extract_day_and_time).str[0]
df_split2['시간'] = df_split2['요일시간'].apply(extract_day_and_time).str[1]

df_split2.drop(columns=["요일시간"], inplace=True)

df_split2

번호   요일        시간
0        1  NaN       NaN
1        2    월   4.0-5.0
2        2    수   3.0-4.0
3        3    월  7.0-11.5
4        4    월   4.0-5.0
...    ...  ...       ...
3770  2616  NaN       NaN
3771  2617  NaN       NaN
3772  2618  NaN       NaN
3773  2619    월   9.0-9.5
3774  2620    화   9.0-9.5

[3775 rows x 3 columns]

In [15]:
df_split2[['시작시간', '종료시간']] = df_split2['시간'].str.split('-', n=1, expand=True)

df_split2.drop(columns=['시간'], inplace=True)

df_split2

번호   요일 시작시간  종료시간
0        1  NaN  NaN   NaN
1        2    월  4.0   5.0
2        2    수  3.0   4.0
3        3    월  7.0  11.5
4        4    월  4.0   5.0
...    ...  ...  ...   ...
3770  2616  NaN  NaN   NaN
3771  2617  NaN  NaN   NaN
3772  2618  NaN  NaN   NaN
3773  2619    월  9.0   9.5
3774  2620    화  9.0   9.5

[3775 rows x 4 columns]

In [16]:
df.drop(columns=['요일/시간', '강의실'], inplace=True)

df

번호   교과과정     교과영역구분     학수코드 학수번호              교과목명  교원명 수업캠퍼스  학점  \
0        1   학문기초  제4영역:자연과학  PRI4001   01          미적분학및연습1  문환표    서울   3   
1        2   학문기초  제4영역:자연과학  PRI4001   02          미적분학및연습1  박장희    서울   3   
2        3   학문기초  제4영역:자연과학  PRI4002   01         일반물리학및실험1  조훈영    서울   4   
3        4   학문기초  제4영역:자연과학  PRI4012   01          미적분학및연습2  엄건애    서울   3   
4        5   학문기초  제4영역:자연과학  PRI4012   02          미적분학및연습2  최광호    서울   3   
...    ...    ...        ...      ...  ...               ...  ...   ...  ..   
2615  2616  나노디그리         이론  NAD1007   01         N-딥러닝알고리즘  김지희   NaN   1   
2616  2617  나노디그리         이론  NAD1008   01     N-추천시스템기본알고리즘  김지희   NaN   1   
2617  2618  나노디그리         이론  NAD1009   01     N-추천시스템고급알고리즘  김지희   NaN   1   
2618  2619  나노디그리        캡스톤  NAD3001   01  N-데이터사이언스캡스톤프로젝트  김광일   NaN   1   
2619  2620  나노디그리        캡스톤  NAD3004   01    N-추천시스템캡스톤프로젝트  김광일   NaN   1   

         개설대학        개설학과/전공  \
0        이과대학       이과대학 수학과   
1        이과대학       이과대학 수학과   
2        이과대학  이과대학 물리반도체과학부   
3        이과대학       이과대학 수학과   
4        이과대학       이과대학 수학과   
...       ...            ...   
2615  행정및부속기관   부속교육기관 SW교육원   
2616  행정및부속기관   부속교육기관 SW교육원   
2617  행정및부속기관   부속교육기관 SW교육원   
2618  행정및부속기관   부속교육기관 SW교육원   
2619  행정및부속기관   부속교육기관 SW교육원   

                                                     비고  
0                                                 사이버강의  
1                                                   NaN  
2                                         팀티칭, 수강정원 50명  
3                                                   NaN  
4                                      공과대 일괄수강신청 예정 강좌  
...                                                 ...  
2615            사이버강의, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, P/F  
2616            사이버강의, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, P/F  
2617            사이버강의, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, P/F  
2618  실습, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, 해당 나노디그리 전체 교...  
2619  실습, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, 해당 나노디그리 전체 교...  

[2620 rows x 12 columns]

컬럼 이름 변경(DB 테이블 컬럼명으로 변경)

In [17]:
df_rename_dicts = {
    "번호" : "lecture_id",
    "교과과정" : "lecture_curriculum",
    "교과영역구분" : "lecture_classification",
    "학수코드" : "lecture_code",
    "학수번호" : "lecture_number",
    "교과목명" : "lecture_name",
    "교원명" : "lecture_professor",
    "수업캠퍼스" : "lecture_campus",
    "학점" : "lecture_credit",
    "개설대학" : "lecture_univ",
    "개설학과/전공" : "lecture_major",
    "비고" : "lecture_remark",
}

df.rename(columns=df_rename_dicts, inplace=True)

df

lecture_id lecture_curriculum lecture_classification lecture_code  \
0              1               학문기초              제4영역:자연과학      PRI4001   
1              2               학문기초              제4영역:자연과학      PRI4001   
2              3               학문기초              제4영역:자연과학      PRI4002   
3              4               학문기초              제4영역:자연과학      PRI4012   
4              5               학문기초              제4영역:자연과학      PRI4012   
...          ...                ...                    ...          ...   
2615        2616              나노디그리                     이론      NAD1007   
2616        2617              나노디그리                     이론      NAD1008   
2617        2618              나노디그리                     이론      NAD1009   
2618        2619              나노디그리                    캡스톤      NAD3001   
2619        2620              나노디그리                    캡스톤      NAD3004   

     lecture_number      lecture_name lecture_professor lecture_campus  \
0                01          미적분학및연습1               문환표             서울   
1                02          미적분학및연습1               박장희             서울   
2                01         일반물리학및실험1               조훈영             서울   
3                01          미적분학및연습2               엄건애             서울   
4                02          미적분학및연습2               최광호             서울   
...             ...               ...               ...            ...   
2615             01         N-딥러닝알고리즘               김지희            NaN   
2616             01     N-추천시스템기본알고리즘               김지희            NaN   
2617             01     N-추천시스템고급알고리즘               김지희            NaN   
2618             01  N-데이터사이언스캡스톤프로젝트               김광일            NaN   
2619             01    N-추천시스템캡스톤프로젝트               김광일            NaN   

      lecture_credit lecture_univ  lecture_major  \
0                  3         이과대학       이과대학 수학과   
1                  3         이과대학       이과대학 수학과   
2                  4         이과대학  이과대학 물리반도체과학부   
3                  3         이과대학       이과대학 수학과   
4                  3         이과대학       이과대학 수학과   
...              ...          ...            ...   
2615               1      행정및부속기관   부속교육기관 SW교육원   
2616               1      행정및부속기관   부속교육기관 SW교육원   
2617               1      행정및부속기관   부속교육기관 SW교육원   
2618               1      행정및부속기관   부속교육기관 SW교육원   
2619               1      행정및부속기관   부속교육기관 SW교육원   

                                         lecture_remark  
0                                                 사이버강의  
1                                                   NaN  
2                                         팀티칭, 수강정원 50명  
3                                                   NaN  
4                                      공과대 일괄수강신청 예정 강좌  
...                                                 ...  
2615            사이버강의, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, P/F  
2616            사이버강의, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, P/F  
2617            사이버강의, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, P/F  
2618  실습, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, 해당 나노디그리 전체 교...  
2619  실습, 나노디그리 재학생(나노디그리 신청 후 합격자)전용, 해당 나노디그리 전체 교...  

[2620 rows x 12 columns]

In [18]:
df_split_rename_dicts = {
    "번호" : "lecture_id_id",
    "강의실" : "lecture_room"
}

df_split.rename(columns=df_split_rename_dicts, inplace=True)

df_split

lecture_id_id                  lecture_room
0                 1                           NaN
1                 2  102-352(학술/문화관 K352 352 강의실)
2                 2  102-352(학술/문화관 K352 352 강의실)
3                 3         307-309(명진관 A309 강의실)
4                 4      407-109(정보문화관P P304 강의실)
...             ...                           ...
3990           2616                           NaN
3991           2617                           NaN
3992           2618                           NaN
3993           2619   408-229(정보문화관Q Q202 컴퓨터실습실)
3994           2620   408-229(정보문화관Q Q202 컴퓨터실습실)

[3995 rows x 2 columns]

In [19]:
df_split2_rename_dicts = {
    "번호" : "lecture_id_id",
    "요일" : "lecture_day",
    "시작시간" : "lecture_start_time",
    "종료시간" : "lecture_end_time"  
}

df_split2.rename(columns=df_split2_rename_dicts, inplace=True)

df_split2

lecture_id_id lecture_day lecture_start_time lecture_end_time
0                 1         NaN                NaN              NaN
1                 2           월                4.0              5.0
2                 2           수                3.0              4.0
3                 3           월                7.0             11.5
4                 4           월                4.0              5.0
...             ...         ...                ...              ...
3770           2616         NaN                NaN              NaN
3771           2617         NaN                NaN              NaN
3772           2618         NaN                NaN              NaN
3773           2619           월                9.0              9.5
3774           2620           화                9.0              9.5

[3775 rows x 4 columns]

df를 DB 테이블에 연결

In [30]:

db_config = {
    'user' : 'root',
    'password' : 'chldud3366',
    'host' : 'localhost',
    'port' : 3306,
    'name' : 'selfproject'
}

In [31]:
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['name']}")

In [32]:
df.to_sql('myapp_lecture', con=engine, if_exists='append', index=False)

2620

In [33]:
df_split.to_sql('myapp_lectureroom', con=engine, if_exists='append', index=False)

3995

In [34]:
df_split2.to_sql('myapp_lecturetime', con=engine, if_exists='append', index=False)

3775

In [35]:
engine.dispose()